In [3]:
from zipfile import ZipFile
zip = ZipFile('Resources/archive.zip')
zip.extractall('Resources')

In [4]:
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [12]:
#Display all the columns (to see which to drop)
pd.set_option("display.max_columns", None)

## Read In CSV data for Mutual Fund prices A-Z


In [13]:
#Read in CSV data 
#MutualFund prices A-E
df_AE= pd.read_csv(
    Path("Resources/MutualFund Prices - A-E.csv")
)
df_AE.head()

,fund_symbol,price_date,nav_per_share
0,AAAAX,2007-07-31,10.02
1,AAAAX,2007-08-01,9.98
2,AAAAX,2007-08-02,10.01
3,AAAAX,2007-08-03,9.90
4,AAAAX,2007-08-06,9.93


In [14]:
#Read in CSV data 
#MutualFund prices F-K
df_FK= pd.read_csv(
    Path("Resources/MutualFund Prices - F-K.csv")
)
df_FK.head()

,fund_symbol,price_date,nav_per_share
0,FAAAX,2013-11-20,10.08
1,FAAAX,2013-11-21,10.13
2,FAAAX,2013-11-22,10.17
3,FAAAX,2013-11-25,10.18
4,FAAAX,2013-11-26,10.20


In [15]:
#Read in CSV data 
#MutualFund prices L-P
df_LP= pd.read_csv(
    Path("Resources/MutualFund Prices - L-P.csv")
)
df_LP.head()

,fund_symbol,price_date,nav_per_share
0,LAACX,2013-09-03,10.00
1,LAACX,2013-09-04,10.05
2,LAACX,2013-09-05,9.99
3,LAACX,2013-09-06,10.06
4,LAACX,2013-09-09,10.15


In [16]:
#Read in CSV data 
#MutualFund prices Q-Z
df_QZ= pd.read_csv(
    Path("Resources/MutualFund Prices - Q-Z.csv")
)
df_QZ.head()

,fund_symbol,price_date,nav_per_share
0,QAACX,2003-02-12,10.03
1,QAACX,2003-02-13,10.03
2,QAACX,2003-02-14,10.18
3,QAACX,2003-02-18,10.35
4,QAACX,2003-02-19,10.28


In [17]:
#Concatonate dataframes
mutual_fund_df= pd.concat([df_AE,df_FK,df_LP,df_QZ])
mutual_fund_df.shape 

(75657739, 3)

In [18]:
mutual_fund_df.dtypes

fund_symbol       object
price_date        object
nav_per_share    float64
dtype: object

In [19]:
mutual_fund_df["price_date"]=pd.to_datetime(mutual_fund_df['price_date'])

In [20]:
mutual_fund_df.dtypes

fund_symbol              object
price_date       datetime64[ns]
nav_per_share           float64
dtype: object

# Read in Mutual Fund informaiton csv


In [21]:
#Read in CSV data 
#MutualFunds
mutualFunds= pd.read_csv(
    Path("Resources/MutualFunds.csv")
)
mutualFunds.head()

,fund_symbol,quote_type,region,fund_short_name,fund_long_name,currency,initial_investment,subsequent_investment,fund_category,fund_family,exchange_code,exchange_name,exchange_timezone,management_name,management_bio,management_start_date,total_net_assets,year_to_date_return,day50_moving_average,day200_moving_average,week52_high_low_change,week52_high_low_change_perc,week52_high,week52_high_change,week52_high_change_perc,week52_low,week52_low_change,week52_low_change_perc,investment_strategy,fund_yield,morningstar_overall_rating,morningstar_risk_rating,inception_date,last_dividend,last_cap_gain,annual_holdings_turnover,investment_type,size_type,fund_annual_report_net_expense_ratio,category_annual_report_net_expense_ratio,fund_prospectus_net_expense_ratio,fund_prospectus_gross_expense_ratio,fund_max_12b1_fee,fund_max_front_end_sales_load,category_max_front_end_sales_load,fund_max_deferred_sales_load,category_max_deferred_sales_load,fund_year3_expense_projection,fund_year5_expense_projection,fund_year10_expense_projection,asset_cash,asset_stocks,asset_bonds,asset_others,asset_preferred,asset_convertible,fund_sector_basic_materials,fund_sector_communication_services,fund_sector_consumer_cyclical,fund_sector_consumer_defensive,fund_sector_energy,fund_sector_financial_services,fund_sector_healthcare,fund_sector_industrials,fund_sector_real_estate,fund_sector_technology,fund_sector_utilities,fund_price_book_ratio,category_price_book_ratio,fund_price_cashflow_ratio,category_price_cashflow_ratio,fund_price_earning_ratio,category_price_earning_ratio,fund_price_sales_ratio,category_price_sales_ratio,fund_median_market_cap,category_median_market_cap,fund_year3_earnings_growth,category_year3_earnings_growth,fund_bond_maturity,category_bond_maturity,fund_bond_duration,category_bond_duration,fund_bonds_us_government,fund_bonds_aaa,fund_bonds_aa,fund_bonds_a,fund_bonds_bbb,fund_bonds_bb,fund_bonds_b,fund_bonds_below_b,fund_bonds_others,top10_holdings,top10_holdings_total_assets,morningstar_return_rating,returns_as_of_date,fund_return_ytd,category_return_ytd,fund_return_1month,category_return_1month,fund_return_3months,category_return_3months,fund_return_1year,category_return_1year,fund_return_3years,category_return_3years,fund_return_5years,category_return_5years,fund_return_10years,category_return_10years,fund_return_last_bull_market,category_return_last_bull_market,fund_return_last_bear_market,category_return_last_bear_market,years_up,years_down,fund_return_2020,category_return_2020,fund_return_2019,category_return_2019,fund_return_2018,category_return_2018,fund_return_2017,category_return_2017,fund_return_2016,category_return_2016,fund_return_2015,category_return_2015,fund_return_2014,category_return_2014,fund_return_2013,category_return_2013,fund_return_2012,category_return_2012,fund_return_2011,category_return_2011,fund_return_2010,category_return_2010,fund_return_2009,category_return_2009,fund_return_2008,category_return_2008,fund_return_2007,category_return_2007,fund_return_2006,category_return_2006,fund_return_2005,category_return_2005,fund_return_2004,category_return_2004,fund_return_2003,category_return_2003,fund_return_2002,category_return_2002,fund_return_2001,category_return_2001,fund_return_2000,category_return_2000,quarters_up,quarters_down,fund_return_2021_q3,fund_return_2021_q2,fund_return_2021_q1,fund_return_2020_q4,fund_return_2020_q3,fund_return_2020_q2,fund_return_2020_q1,fund_return_2019_q4,fund_return_2019_q3,fund_return_2019_q2,fund_return_2019_q1,fund_return_2018_q4,fund_return_2018_q3,fund_return_2018_q2,fund_return_2018_q1,fund_return_2017_q4,fund_return_2017_q3,fund_return_2017_q2,fund_return_2017_q1,fund_return_2016_q4,fund_return_2016_q3,fund_return_2016_q2,fund_return_2016_q1,fund_return_2015_q4,fund_return_2015_q3,fund_return_2015_q2,fund_return_2015_q1,fund_return_2014_q4,fund_return_2014_q3,fund_return_2014_q2,fund_return_2014_q1,fund_return_2013_q4,fund_return_2013_q3,fund_return_2013_q2,fund_return_2013_q1,f

In [12]:
# Find columns with only 1 value to drop
mutualFunds.loc[: , mutualFunds.dtypes== "object"].nunique()

fund_symbol              23783
quote_type                   1
region                       1
fund_short_name           3044
fund_long_name            6644
currency                     1
fund_category              119
fund_family                310
exchange_code                1
exchange_name                1
exchange_timezone            1
management_name           1388
management_bio            1300
management_start_date     2316
investment_strategy       2245
inception_date            4499
investment_type              3
size_type                    3
top10_holdings            2347
returns_as_of_date          51
esg_peer_group              86
dtype: int64

In [13]:
mutualFunds_counts= mutualFunds.loc[: ,mutualFunds.dtypes=="object"].nunique()
mutualFunds_counts_one= mutualFunds_counts[mutualFunds_counts == 1].index.to_list()
print(mutualFunds_counts_one)

['quote_type', 'region', 'currency', 'exchange_code', 'exchange_name', 'exchange_timezone']


In [14]:
#Drop columns with only 1 unique value
mutualFunds.drop(columns=mutualFunds_counts_one,inplace=True)
mutualFunds

,fund_symbol,fund_short_name,fund_long_name,initial_investment,subsequent_investment,fund_category,fund_family,management_name,management_bio,management_start_date,...,peer_environment_avg,peer_environment_max,social_score,peer_social_min,peer_social_avg,peer_social_max,governance_score,peer_governance_min,peer_governance_avg,peer_governance_max
0,AAAAX,DWS RREEF Real Assets Fund - Cl,DWS RREEF Real Assets Fund - Class A,1000.0,50.0,World Allocation,DWS,John Vojticek,Co-Head of Liquid Real Assets / Chief Investme...,2015-01-15,...,5.05,10.58,7.43,5.98,9.07,11.30,5.43,4.26,7.14,8.11
1,AAAEX,AllianzGI Health Sciences Fund,Virtus AllianzGI Health Sciences Fund Class P,1000000.0,NaN,Health,Virtus,Christopher Chin,NaN,2020-08-27,...,1.43,3.27,12.96,9.52,12.87,15.08,8.40,4.96,7.68,10.30
2,AAAFX,NaN,American Century One Choice Blend+ 2015 Portfo...,2500.0,50.0,Target-Date 2015,American Century Investments,Scott A. Wilson,"Mr. Wilson, Vice President and Portfolio Manag...",2021-03-10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAAGX,Thrivent Large Cap Growth Fund,Thrivent Large Cap Growth Fund Class A,2000.0,50.0,Large Growth,Thrivent Funds,Lauri Brunner,Ms. Brunner has been with Thrivent Financial s...,2018-09-30,...,2.70,5.81,10.13,7.25,10.14,11.97,8.03,5.30,7.54,8.90
4,AAAHX,NaN,American Century One Choice Blend+ 2015 Portfo...,5000000.0,NaN,Target-Date 2015,American Century Investments,Scott A. Wilson,"Mr. Wilson, Vice President and Portfolio Manag...",2021-03-10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23778,ZVNIX,Fidelity Advisor Small Cap Valu,Fidelity Advisor Small Cap Value Fund Class M,50000.0,500.0,Large Growth,Fidelity Investments,Derek Janssen,Derek Janssen is portfolio manager of the Fide...,2015-08-31,...,2.70,5.81,10.53,7.25,10.14,11.97,7.25,5.30,7.54,8.90
23779,VHYAX,Capital World Bond Fund - Class,American Funds Capital World Bond Fund Class 5...,3000.0,1.0,Large Value,American Funds,Thomas HÃ¸gh,Thomas H. HÃ¸gh is a fixed income portfolio ma...,2016-02-25,...,5.00,9.11,10.83,7.04,10.48,12.42,7.84,5.83,7.71,9.30
23780,VIAAX,Capital World Growth and Income,American Funds Capital World Growth and Income...,3000.0,1.0,Foreign Large Growth,American Funds,Michael Alfonso Cohen,Michael Cohen is an equity portfolio manager a...,2016-02-25,...,3.87,6.22,8.53,7.25,8.96,11.96,7.70,5.79,7.42,8.78
23781,VIHAX,Templeton China World Cl R6,Templeton China World Fund Class R6,3000.0,1.0,Foreign Large Value,Franklin Templeton Investments,Michael B. Lai,He joined Franklin Templeton in August 2019. P...,2016-02-25,...,5.35,11.49,9.55,6.59,9.24,11.66,8.79,5.79,8.10,10.85


We noticed that there are many `NaN` values throughout the data set. We will now explore which columns/rows have NaN and will remove an appropriate amount of columns/rows.

In [93]:
mutualFunds.dropna(axis = 0)

,fund_symbol,fund_short_name,fund_long_name,initial_investment,subsequent_investment,fund_category,fund_family,management_name,management_bio,management_start_date,...,peer_environment_avg,peer_environment_max,social_score,peer_social_min,peer_social_avg,peer_social_max,governance_score,peer_governance_min,peer_governance_avg,peer_governance_max


If we drop rows that have an `NaN`, then all data is removed. Instead, let's look at the columns that have missing data.

In [44]:
# Looking at each column and the count of NaN in each column
list = []
for index, row in pd.DataFrame(mutualFunds.isna().sum()).iterrows():
    list.append((index,row))


In [96]:
list[0]

('fund_symbol',
 0    0
 Name: fund_symbol, dtype: int64)

We plan to find the number of columns that have above a certain percentage of `NaN` and then remove those columns. We have chosen 60% so far.

In [99]:
threshold = 0.6*len(mutualFunds)
drop_columns = []
for i in range(len(list)):
    if list[i][1][0] >= threshold:
        drop_columns.append(list[i][0])

In [100]:
print(len(drop_columns))
drop_columns


49


['subsequent_investment',
 'last_cap_gain',
 'fund_max_front_end_sales_load',
 'fund_max_deferred_sales_load',
 'fund_bond_maturity',
 'fund_return_2008',
 'fund_return_2007',
 'fund_return_2006',
 'fund_return_2005',
 'fund_return_2004',
 'fund_return_2003',
 'fund_return_2002',
 'fund_return_2001',
 'fund_return_2000',
 'fund_return_2021_q3',
 'fund_return_2021_q2',
 'fund_return_2008_q4',
 'fund_return_2007_q4',
 'fund_return_2007_q3',
 'fund_return_2007_q2',
 'fund_return_2007_q1',
 'fund_return_2006_q4',
 'fund_return_2006_q3',
 'fund_return_2006_q2',
 'fund_return_2006_q1',
 'fund_return_2005_q4',
 'fund_return_2005_q3',
 'fund_return_2005_q2',
 'fund_return_2005_q1',
 'fund_return_2004_q4',
 'fund_return_2004_q3',
 'fund_return_2004_q2',
 'fund_return_2004_q1',
 'fund_return_2003_q4',
 'fund_return_2003_q3',
 'fund_return_2003_q2',
 'fund_return_2003_q1',
 'fund_return_2002_q4',
 'fund_return_2002_q3',
 'fund_return_2002_q2',
 'fund_return_2002_q1',
 'fund_return_2001_q4',
 'fun

In [101]:
mutualFunds.drop(columns=drop_columns, inplace = True)
mutualFunds

,fund_symbol,fund_short_name,fund_long_name,initial_investment,fund_category,fund_family,management_name,management_bio,management_start_date,total_net_assets,...,peer_environment_avg,peer_environment_max,social_score,peer_social_min,peer_social_avg,peer_social_max,governance_score,peer_governance_min,peer_governance_avg,peer_governance_max
0,AAAAX,DWS RREEF Real Assets Fund - Cl,DWS RREEF Real Assets Fund - Class A,1000.0,World Allocation,DWS,John Vojticek,Co-Head of Liquid Real Assets / Chief Investme...,2015-01-15,2.979347e+09,...,5.05,10.58,7.43,5.98,9.07,11.30,5.43,4.26,7.14,8.11
1,AAAEX,AllianzGI Health Sciences Fund,Virtus AllianzGI Health Sciences Fund Class P,1000000.0,Health,Virtus,Christopher Chin,NaN,2020-08-27,1.953489e+08,...,1.43,3.27,12.96,9.52,12.87,15.08,8.40,4.96,7.68,10.30
2,AAAFX,NaN,American Century One Choice Blend+ 2015 Portfo...,2500.0,Target-Date 2015,American Century Investments,Scott A. Wilson,"Mr. Wilson, Vice President and Portfolio Manag...",2021-03-10,2.594600e+04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAAGX,Thrivent Large Cap Growth Fund,Thrivent Large Cap Growth Fund Class A,2000.0,Large Growth,Thrivent Funds,Lauri Brunner,Ms. Brunner has been with Thrivent Financial s...,2018-09-30,2.078607e+09,...,2.70,5.81,10.13,7.25,10.14,11.97,8.03,5.30,7.54,8.90
4,AAAHX,NaN,American Century One Choice Blend+ 2015 Portfo...,5000000.0,Target-Date 2015,American Century Investments,Scott A. Wilson,"Mr. Wilson, Vice President and Portfolio Manag...",2021-03-10,2.594600e+04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23778,ZVNIX,Fidelity Advisor Small Cap Valu,Fidelity Advisor Small Cap Value Fund Class M,50000.0,Large Growth,Fidelity Investments,Derek Janssen,Derek Janssen is portfolio manager of the Fide...,2015-08-31,1.222816e+08,...,2.70,5.81,10.53,7.25,10.14,11.97,7.25,5.30,7.54,8.90
23779,VHYAX,Capital World Bond Fund - Class,American Funds Capital World Bond Fund Class 5...,3000.0,Large Value,American Funds,Thomas HÃ¸gh,Thomas H. HÃ¸gh is a fixed income portfolio ma...,2016-02-25,4.847444e+10,...,5.00,9.11,10.83,7.04,10.48,12.42,7.84,5.83,7.71,9.30
23780,VIAAX,Capital World Growth and Income,American Funds Capital World Growth and Income...,3000.0,Foreign Large Growth,American Funds,Michael Alfonso Cohen,Michael Cohen is an equity portfolio manager a...,2016-02-25,4.288452e+09,...,3.87,6.22,8.53,7.25,8.96,11.96,7.70,5.79,7.42,8.78
23781,VIHAX,Templeton China World Cl R6,Templeton China World Fund Class R6,3000.0,Foreign Large Value,Franklin Templeton Investments,Michael B. Lai,He joined Franklin Templeton in August 2019. P...,2016-02-25,3.242179e+09,...,5.35,11.49,9.55,6.59,9.24,11.66,8.79,5.79,8.10,10.85


In [102]:
mutualFunds.dropna()

,fund_symbol,fund_short_name,fund_long_name,initial_investment,fund_category,fund_family,management_name,management_bio,management_start_date,total_net_assets,...,peer_environment_avg,peer_environment_max,social_score,peer_social_min,peer_social_avg,peer_social_max,governance_score,peer_governance_min,peer_governance_avg,peer_governance_max
14,AAARX,American Century Strategic Allo,American Century Strategic Allocation: Aggress...,2500.0,Allocation--70% to 85% Equity,American Century Investments,Scott A. Wilson,"Mr. Wilson, Vice President and Portfolio Manag...",2011-04-01,8.483190e+08,...,4.16,6.70,9.01,4.74,9.43,10.84,6.82,3.91,7.15,9.10
54,AACTX,American Fds 2020 Target Date R,American Funds 2020 Target Date Retirement Fun...,250.0,Target-Date 2020,American Funds,Wesley K.-S. Phoa,Wesley K. Phoa is a solutions portfolio manage...,2012-01-01,1.901028e+10,...,4.09,4.68,11.14,9.10,10.13,11.14,7.48,6.83,7.43,7.73
59,AADAX,Invesco Select Risk: Growth Inv,Invesco Select Risk: Growth Investor Fund Class A,1000.0,Allocation--70% to 85% Equity,Invesco,Jeffrey Bennett,Jeff Bennett is a Senior Portfolio Manager and...,2020-04-30,1.127792e+09,...,4.16,6.70,7.34,4.74,9.43,10.84,5.71,3.91,7.15,9.10
61,AADCX,Invesco Select Risk: Growth Inv,Invesco Select Risk: Growth Investor Fund Class C,1000.0,Allocation--70% to 85% Equity,Invesco,Jeffrey Bennett,Jeff Bennett is a Senior Portfolio Manager and...,2020-04-30,1.127792e+09,...,4.16,6.70,7.34,4.74,9.43,10.84,5.71,3.91,7.15,9.10
69,AADRX,Invesco Select Risk: Growth Inv,Invesco Select Risk: Growth Investor Fund Class R,1000.0,Allocation--70% to 85% Equity,Invesco,Jeffrey Bennett,Jeff Bennett is a Senior Portfolio Manager and...,2020-04-30,1.127792e+09,...,4.16,6.70,7.34,4.74,9.43,10.84,5.71,3.91,7.15,9.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23212,WBCCX,Eaton Vance Atlanta Capital Sel,Eaton Vance Atlanta Capital Select Equity Fund...,1000.0,Allocation--50% to 70% Equity,Eaton Vance,William O. Bell,"William O. Bell, IV, CFA is a Vice President a...",2019-09-13,5.432181e+07,...,4.11,6.45,11.23,6.86,10.03,11.68,7.63,5.07,7.43,9.59
23224,WBLAX,Mirova Global Sustainable Equit,Mirova Global Sustainable Equity Fund Class A,250.0,Allocation--50% to 70% Equity,Natixis Funds,Amber Fairbanks,"Ms. Fairbanks, CFA, is a Portfolio Manager wit...",2016-10-01,6.773392e+07,...,4.11,6.45,9.94,6.86,10.03,11.68,7.46,5.07,7.43,9.59
23304,WEBAX,Eaton Vance New Jersey Municipa,Eaton Vance New Jersey Municipal Income Fund C...,1000.0,Allocation--50% to 70% Equity,Eaton Vance,Adam A. Weigold,"Adam A. Weigold, Vice President of Boston Mana...",2019-09-13,5.432181e+07,...,4.11,6.45,11.23,6.86,10.03,11.68,7.63,5.07,7.43,9.59
23305,WEBCX,Eventide Multi-Asset Income Fun,Eventide Multi-Asset Income Fund Class N,1000.0,Allocation--50% to 70% Equity,Eventide Funds,David M. Dirk,"David M. Dirk, CFA, is Director of Portfolio M...",2019-09-13,5.432181e+07,...,4.11,6.45,11.23,6.86,10.03,11.68,7.63,5.07,7.43,9.59
